In [11]:
# @title Upload CSV
import pandas as pd
import random

hari_df = pd.read_csv('data_skripsi_hari.csv')
ruang_df = pd.read_csv('data_skripsi_ruang.csv')
jam_df = pd.read_csv('data_skripsi_jam.csv')
dosen_df = pd.read_csv('data_skripsi_dosen.csv')
mk_genap_df = pd.read_csv('data_skripsi_mk_genap.csv')
data_dosen_df = pd.read_csv('data_skripsi_data_dosen.csv')

In [12]:
# @title Gabungkan data_dosen dengan dosen
merged_df = pd.merge(
    pd.merge(data_dosen_df, dosen_df, on='id_dosen'),
    mk_genap_df, on='id_mk_genap'
)

print("Data Gabungan:")
print(merged_df.head())

total_sks = merged_df['sks'].sum()
print(f"Total slot SKS yang dibutuhkan: {total_sks}")
print(f"Total data dalam merged_df: {len(merged_df)}")

Data Gabungan:
   id_dosen  id_mk_genap kelas                   nama_dosen  \
0         2    211840231     A  Ahmad Azhari, S.Kom., M.Eng   
1         2    211861331     A  Ahmad Azhari, S.Kom., M.Eng   
2         2    211861431     A  Ahmad Azhari, S.Kom., M.Eng   
3         3    211840131     A     Ali Tarmuji, S.T., M.Cs.   
4         3    211860330     A     Ali Tarmuji, S.T., M.Cs.   

                              nama_mk_genap  smt  sks    sifat kategori  \
0                          Grafika Komputer    4    3    Wajib        -   
1                Pengembangan Aplikasi Game    6    3  Pilihan       SC   
2                          Pengenalan Pola     6    3  Pilihan       SC   
3  Analisis dan Perancangan Perangkat Lunak    4    3    Wajib        -   
4                  Rekayasa Perangkat Lunak    6    3    Wajib        -   

    metode  
0  Offline  
1  Offline  
2   Online  
3  Offline  
4  Offline  
Total slot SKS yang dibutuhkan: 282
Total data dalam merged_df: 107


Tahapan GWO

1. Preprocessing
    
    Membangun struktur slot waktu.

2. Inisialisasi Populasi (GWO)
    

    Inisialisasi populasi serigala acak (Xi).

    Setiap "serigala" dalam GWO mewakili solusi penjadwalan yang mungkin (misalnya: variasi pengaturan slot).

3. Fitness Function (GWO)
    Hitung nilai fitness untuk setiap serigala.

    Mengevaluasi kualitas penjadwalan (misalnya: minimalisasi konflik, kepadatan ruang, dll).

4. Proses Optimasi (GWO)
    Tentukan Alpha, Beta, dan Delta berdasarkan nilai fitness.

    Menggunakan hierarki Alpha, Beta, Delta untuk memperbarui posisi solusi.

    While (iterasi < maksimum iterasi):
      1. Perbarui parameter a, A, dan C.
      2. For setiap serigala:
          1. Perbarui posisi serigala berdasarkan Alpha, Beta, dan Delta.
      3. Hitung nilai fitness untuk setiap serigala.
      4. Perbarui Alpha, Beta, dan Delta.
5. Postprocessing
    Return solusi terbaik (Alpha).
    
    Menampilkan jadwal terbaik hasil optimasi.


In [13]:
# @title Preprosessing
hari_list = hari_df['nama_hari'].tolist()
ruang_list = ruang_df['nama_ruang'].tolist()
jam_list = jam_df[['id_jam', 'jam_awal', 'jam_akhir']].to_dict('records')
mata_kuliah_list = mk_genap_df.set_index('id_mk_genap').to_dict('index')

def slot_generator():
    wolf = []
    for id_counter, (hari, ruang, jam) in enumerate(
        [(h, r, j) for h in hari_list for r in ruang_list for j in jam_list], start=1
    ):
        wolf.append({
            "id_slot": id_counter,
            "mata_kuliah": None,
            "dosen": None,
            "ruang": ruang,
            "hari": hari,
            "jam_mulai": jam['jam_awal'],
            "jam_selesai": jam['jam_akhir'],
            "kelas": None,
            "sks": None,
            "metode": None
        })
    return wolf

slots = slot_generator()
for slot in slots[:5]:
    print(slot)

print("\n")
for slot in slots[-5:]:
    print(slot)

{'id_slot': 1, 'mata_kuliah': None, 'dosen': None, 'ruang': '4.1.5.55', 'hari': 'Senin', 'jam_mulai': '7:00:00', 'jam_selesai': '7:50:00', 'kelas': None, 'sks': None, 'metode': None}
{'id_slot': 2, 'mata_kuliah': None, 'dosen': None, 'ruang': '4.1.5.55', 'hari': 'Senin', 'jam_mulai': '7:50:00', 'jam_selesai': '8:45:00', 'kelas': None, 'sks': None, 'metode': None}
{'id_slot': 3, 'mata_kuliah': None, 'dosen': None, 'ruang': '4.1.5.55', 'hari': 'Senin', 'jam_mulai': '8:45:00', 'jam_selesai': '9:35:00', 'kelas': None, 'sks': None, 'metode': None}
{'id_slot': 4, 'mata_kuliah': None, 'dosen': None, 'ruang': '4.1.5.55', 'hari': 'Senin', 'jam_mulai': '9:35:00', 'jam_selesai': '10:25:00', 'kelas': None, 'sks': None, 'metode': None}
{'id_slot': 5, 'mata_kuliah': None, 'dosen': None, 'ruang': '4.1.5.55', 'hari': 'Senin', 'jam_mulai': '10:30:00', 'jam_selesai': '11:20:00', 'kelas': None, 'sks': None, 'metode': None}


{'id_slot': 500, 'mata_kuliah': None, 'dosen': None, 'ruang': '4.1.5.63', 'hari'

In [14]:
import random
from collections import defaultdict
from datetime import datetime

# Fungsi bantu: konversi waktu ke menit
def time_to_minutes(t):
    try:
        dt = datetime.strptime(t, "%H:%M:%S")
    except ValueError:
        dt = datetime.strptime(t, "%H:%M")
    return dt.hour * 60 + dt.minute

# Fungsi utama: buat jadwal acak
def create_random_schedule():
    schedule = slot_generator()  # Pastikan fungsi slot_generator() telah didefinisikan
    merged_shuffled = merged_df.sample(frac=1).iterrows()  # Pastikan merged_df telah didefinisikan
    courses_inserted = 0
    # Tracking alokasi, meskipun pengecekan overlap tidak digunakan, tetap disimpan sebagai referensi
    room_allocations = defaultdict(list)    # key: (ruang, hari) -> list of (start, end)
    teacher_allocations = defaultdict(list)  # key: (dosen, hari) -> list of (start, end)
    class_allocations = defaultdict(list)    # key: (kelas, hari) -> list of (start, end)

    for _, row in merged_shuffled:
        mata_kuliah = row['nama_mk_genap']
        dosen = row['nama_dosen']
        kelas = row['kelas']
        sks = int(row['sks'])
        metode = row['metode']

        possible_positions = list(range(len(schedule) - sks + 1))
        random.shuffle(possible_positions)
        candidate_blocks = []

        for i in possible_positions:
            block = schedule[i:i+sks]
            # Pastikan semua slot kosong dan berada di hari yang sama
            if not all(slot['mata_kuliah'] is None for slot in block) or not all(slot['hari'] == block[0]['hari'] for slot in block):
                continue
            # Untuk offline, cek kesamaan ruangan
            if metode != 'Online' and not all(slot['ruang'] == block[0]['ruang'] for slot in block):
                continue
            # Pengecekan waktu berurutan dihapus, sehingga blok diperbolehkan memiliki gap waktu

            hari = block[0]['hari']
            ruang = "Online" if metode == 'Online' else block[0]['ruang']
            time_block = (time_to_minutes(block[0]['jam_mulai']), time_to_minutes(block[-1]['jam_selesai']))

            # Pengecekan overlap dihapus, jadi langsung tambahkan blok kandidat
            kelas_already = len(class_allocations[(kelas, hari)]) > 0
            candidate_blocks.append((block, time_block, kelas_already))

        # Pemilihan blok kandidat: ambil blok pertama jika ada kandidat
        selected_block = candidate_blocks[0][0] if candidate_blocks else None

        if selected_block is None:
            print(f"Gagal menempatkan: {kelas} - {mata_kuliah} - {dosen}")
        else:
            for slot in selected_block:
                slot.update({
                    "mata_kuliah": mata_kuliah,
                    "dosen": dosen,
                    "kelas": kelas,
                    "sks": sks,
                    "metode": metode,
                    "ruang": "Online" if metode == 'Online' else slot['ruang']
                })
            hari = selected_block[0]['hari']
            ruang = "Online" if metode == 'Online' else selected_block[0]['ruang']
            time_block = (time_to_minutes(selected_block[0]['jam_mulai']),
                          time_to_minutes(selected_block[-1]['jam_selesai']))

            if metode != 'Online':
                room_allocations[(ruang, hari)].append(time_block)
            teacher_allocations[(dosen, hari)].append(time_block)
            class_allocations[(kelas, hari)].append(time_block)
            courses_inserted += 1

    # print(f"\nTotal mata kuliah yang berhasil dimasukkan: {courses_inserted} / {len(merged_df)}")
    return schedule

# Contoh penggunaan:
population_size = 1
population = [create_random_schedule() for _ in range(population_size)]

for schedule in population:
    for slot in schedule:
        print(slot)

{'id_slot': 1, 'mata_kuliah': 'Matematika Diskrit', 'dosen': 'Dewi Soyusiawaty, S.T., M.T', 'ruang': '4.1.5.55', 'hari': 'Senin', 'jam_mulai': '7:00:00', 'jam_selesai': '7:50:00', 'kelas': 'A', 'sks': 3, 'metode': 'Offline'}
{'id_slot': 2, 'mata_kuliah': 'Matematika Diskrit', 'dosen': 'Dewi Soyusiawaty, S.T., M.T', 'ruang': '4.1.5.55', 'hari': 'Senin', 'jam_mulai': '7:50:00', 'jam_selesai': '8:45:00', 'kelas': 'A', 'sks': 3, 'metode': 'Offline'}
{'id_slot': 3, 'mata_kuliah': 'Matematika Diskrit', 'dosen': 'Dewi Soyusiawaty, S.T., M.T', 'ruang': '4.1.5.55', 'hari': 'Senin', 'jam_mulai': '8:45:00', 'jam_selesai': '9:35:00', 'kelas': 'A', 'sks': 3, 'metode': 'Offline'}
{'id_slot': 4, 'mata_kuliah': None, 'dosen': None, 'ruang': '4.1.5.55', 'hari': 'Senin', 'jam_mulai': '9:35:00', 'jam_selesai': '10:25:00', 'kelas': None, 'sks': None, 'metode': None}
{'id_slot': 5, 'mata_kuliah': 'Manajemen Proyek Teknologi Informasi', 'dosen': 'Guntur  Maulana Zamroni, B.Sc., M.Kom.', 'ruang': 'Online', '

Konstrain berat 1
Konstrain ringan 0.5

Konstrain berat:
dosen tidak boleh mengajar mata kuliah/kelas berbeda pada jam yang sama
ruang kelas tidak boleh digunakan lebih dari 1 kelas/dosen
3 sks membutuhkan 3 slot waktu
ruangan antar kelas harus sama
slot waktu harus berurutan

Konstrain ringan:
kelas dijadwalkan pada waktu paling akhir dan paling awal pada hari itu

In [15]:
def calculate_fitness(schedule):
    penalty = 0

    teacher_conflicts = []              # Hard constraint: dosen mengajar dua kelas bersamaan
    room_conflicts = []                 # Hard constraint: ruangan dipakai lebih dari satu kelas bersamaan
    
    teacher_preference_conflicts = []   # Soft constraint: preferensi dosen
    saturday_conflicts = []             # Soft constraint: hari Sabtu

    # Tracking hard constraint
    teacher_intervals = defaultdict(list)
    room_intervals = defaultdict(list)

    for slot in schedule:
        if not slot['mata_kuliah']:
            continue

        day = slot['hari']
        teacher = slot['dosen']
        room = slot['ruang']
        kelas = slot['kelas']
        id_slot = slot['id_slot']
        start = time_to_minutes(slot['jam_mulai'])
        end = time_to_minutes(slot['jam_selesai'])
        
        teacher_intervals[(teacher, day)].append((start, end, id_slot))
        if room != "Online":
            room_intervals[(room, day)].append((start, end, id_slot))
        
        # Soft constraint: preferensi dosen
        if str(teacher) in ["Ardiansyah, Dr., S.T., M.Cs."]:
            if start <= 720:
                teacher_preference_conflicts.append({
                    'teacher': teacher,
                    'id_slot': id_slot,
                    'jam': slot['jam_mulai'],
                    'message': "Hanya ingin dijadwalkan pada jam 12 atau lebih"
                })
                penalty += 0.5

        # Soft constraint: setiap slot pada hari Sabtu
        if day.lower() == "sabtu":
            saturday_conflicts.append({
                'id_slot': id_slot,
                'teacher': teacher,
                'kelas': kelas,
                'message': "Kelas dijadwalkan pada hari Sabtu"
            })
            penalty += 0.5

    # Hard constraint: Dosen
    for (teacher, day), intervals in teacher_intervals.items():
        intervals.sort(key=lambda x: x[0])
        for i in range(1, len(intervals)):
            prev = intervals[i-1]
            curr = intervals[i]
            if curr[0] < prev[1]:
                teacher_conflicts.append({
                    'teacher': teacher,
                    'conflict_ids': (prev[2], curr[2])
                })
                penalty += 1

    # Hard constraint: Ruangan
    for (room, day), intervals in room_intervals.items():
        intervals.sort(key=lambda x: x[0])
        for i in range(1, len(intervals)):
            prev = intervals[i-1]
            curr = intervals[i]
            if curr[0] < prev[1]:
                room_conflicts.append({
                    'room': room,
                    'day': day,
                    'conflict_ids': (prev[2], curr[2]),
                    'messages': "dipakai lebih dari 1 kelas"
                })
                penalty += 1
    
    return {
        'penalty': penalty,
        'violations': {
            'hard_teacher': len(teacher_conflicts),
            'hard_room': len(room_conflicts),
            'teacher_preference': len(teacher_preference_conflicts),
            'saturday': len(saturday_conflicts)
        },
        'teacher_conflicts': teacher_conflicts,
        'room_conflicts': room_conflicts,
        'teacher_preference_conflicts': teacher_preference_conflicts,
        'saturday_conflicts': saturday_conflicts
    }

# validasi ruangan
# def compare_room_usage(schedule):
#     room_usage = defaultdict(list)
    
#     for slot in schedule:
#         if not slot['mata_kuliah']:
#             continue
#         room = slot['ruang']
#         usage_info = {
#             'id_slot': slot['id_slot'],
#             'dosen': slot['dosen'],
#             'kelas': slot['kelas'],
#             'hari': slot['hari'],
#             'jam_mulai': slot['jam_mulai'],
#             'jam_selesai': slot['jam_selesai']
#         }
#         room_usage[room].append(usage_info)
    
#     for room in room_usage:
#         room_usage[room].sort(key=lambda x: (x['hari'], time_to_minutes(x['jam_mulai'])))
#     return room_usage

population_size = 1
population = [create_random_schedule() for _ in range(population_size)]

for idx, schedule in enumerate(population):
    fitness = calculate_fitness(schedule)
    print(f"\nIndividu {idx+1}:")
    for slot in schedule:
        print(slot)

    print(f"\nTotal Penalty: {fitness['penalty']}")
    
    # validasi konflik dosen (hard constraint)
    print("\nDetail Konflik Dosen:")
    if not fitness['teacher_conflicts']:
        print(" - Tidak ada konflik dosen.")
    else:
        for conflict in fitness['teacher_conflicts']:
            conflict_ids = conflict['conflict_ids']
            print(f" - {conflict['teacher']}: Konflik pada Slot {conflict_ids[0]} dan {conflict_ids[1]}")
    
    # validasi ruangan (hard constraint)
    print("\nValidasi Data Ruangan:")
    room_conflicts = fitness['room_conflicts']
    if not room_conflicts:
        print(" - Data ruangan valid, tidak ada konflik.")
    else:
        for conflict in room_conflicts:
            conflict_ids = conflict['conflict_ids']
            print(f" - Ruang {conflict['room']} {conflict['message']} : Konflik pada Slot {conflict_ids[0]} dan {conflict_ids[1]}")

    # Soft constraint: preferensi dosen
    print("\nSoft Constraint - Preferensi Dosen (Jam 6 sampai 12):")
    if not fitness['teacher_preference_conflicts']:
        print(" - Tidak ada konflik preferensi dosen.")
    else:
        for conflict in fitness['teacher_preference_conflicts']:
            print(f" - Slot {conflict['id_slot']} - {conflict['teacher']} (jam mulai: {conflict['jam']}): {conflict['message']}")

    # Soft constraint: hari Sabtu
    print("\nSoft Constraint - Hari Sabtu:")
    if not fitness['saturday_conflicts']:
        print(" - Tidak ada kelas pada hari Sabtu.")
    else:
        for conflict in fitness['saturday_conflicts']:
            print(f" - Slot {conflict['id_slot']} - {conflict['teacher']}, Kelas {conflict['kelas']}: {conflict['message']}")

    # Komparasi Penggunaan Ruangan Berdasarkan Ruangan
    # print("\nKomparasi Penggunaan Ruangan Berdasarkan Ruangan:")
    # room_usage = compare_room_usage(schedule)
    # if not room_usage:
    #     print(" - Tidak ada data penggunaan ruangan.")
    # else:
    #     for room, usages in room_usage.items():
    #         print(f" - Ruang: {room}")
    #         for usage in usages:
    #             print(f"    ID Slot: {usage['id_slot']}, Dosen: {usage['dosen']}, Kelas: {usage['kelas']}, Hari: {usage['hari']}, Jam Mulai: {usage['jam_mulai']}, Jam Selesai: {usage['jam_selesai']}")



Individu 1:
{'id_slot': 1, 'mata_kuliah': 'Kriptografi', 'dosen': 'Eko Aribowo, S.T., M.Kom.', 'ruang': '4.1.5.55', 'hari': 'Senin', 'jam_mulai': '7:00:00', 'jam_selesai': '7:50:00', 'kelas': 'A', 'sks': 3, 'metode': 'Offline'}
{'id_slot': 2, 'mata_kuliah': 'Kriptografi', 'dosen': 'Eko Aribowo, S.T., M.Kom.', 'ruang': '4.1.5.55', 'hari': 'Senin', 'jam_mulai': '7:50:00', 'jam_selesai': '8:45:00', 'kelas': 'A', 'sks': 3, 'metode': 'Offline'}
{'id_slot': 3, 'mata_kuliah': 'Kriptografi', 'dosen': 'Eko Aribowo, S.T., M.Kom.', 'ruang': '4.1.5.55', 'hari': 'Senin', 'jam_mulai': '8:45:00', 'jam_selesai': '9:35:00', 'kelas': 'A', 'sks': 3, 'metode': 'Offline'}
{'id_slot': 4, 'mata_kuliah': None, 'dosen': None, 'ruang': '4.1.5.55', 'hari': 'Senin', 'jam_mulai': '9:35:00', 'jam_selesai': '10:25:00', 'kelas': None, 'sks': None, 'metode': None}
{'id_slot': 5, 'mata_kuliah': 'Algoritma Pemrograman', 'dosen': 'Ardiansyah, Dr., S.T., M.Cs.', 'ruang': '4.1.5.55', 'hari': 'Senin', 'jam_mulai': '10:30:0

In [ ]:
def calculate_fitness(schedule):
    penalty = 0

    teacher_conflicts = []              # Hard constraint: dosen mengajar dua kelas bersamaan
    room_conflicts = []                 # Hard constraint: ruangan dipakai lebih dari satu kelas bersamaan
    
    teacher_preference_conflicts = []   # Soft constraint: preferensi dosen
    saturday_conflicts = []             # Soft constraint: hari Sabtu

    # Tracking hard constraint
    teacher_intervals = defaultdict(list)
    room_intervals = defaultdict(list)

    for slot in schedule:
        if not slot['mata_kuliah']:
            continue

        day = slot['hari']
        teacher = slot['dosen']
        room = slot['ruang']
        kelas = slot['kelas']
        id_slot = slot['id_slot']
        start = time_to_minutes(slot['jam_mulai'])
        end = time_to_minutes(slot['jam_selesai'])
        
        teacher_intervals[(teacher, day)].append((start, end, id_slot))
        if room != "Online":
            room_intervals[(room, day)].append((start, end, id_slot))
        
        # Soft constraint: preferensi dosen
        if str(teacher) in ["Ardiansyah, Dr., S.T., M.Cs."]:
            if start <= 720:
                teacher_preference_conflicts.append({
                    'teacher': teacher,
                    'id_slot': id_slot,
                    'jam': slot['jam_mulai'],
                    'message': "Hanya ingin dijadwalkan pada jam 12 atau lebih"
                })
                penalty += 0.5

        # Soft constraint: setiap slot pada hari Sabtu
        if day.lower() == "sabtu":
            saturday_conflicts.append({
                'id_slot': id_slot,
                'teacher': teacher,
                'kelas': kelas,
                'message': "Kelas dijadwalkan pada hari Sabtu"
            })
            penalty += 0.5

    # Hard constraint: Dosen
    for (teacher, day), intervals in teacher_intervals.items():
        intervals.sort(key=lambda x: x[0])
        for i in range(1, len(intervals)):
            prev = intervals[i-1]
            curr = intervals[i]
            if curr[0] < prev[1]:
                teacher_conflicts.append({
                    'teacher': teacher,
                    'conflict_ids': (prev[2], curr[2])
                })
                penalty += 1

    # Hard constraint: Ruangan
    for (room, day), intervals in room_intervals.items():
        intervals.sort(key=lambda x: x[0])
        for i in range(1, len(intervals)):
            prev = intervals[i-1]
            curr = intervals[i]
            if curr[0] < prev[1]:
                room_conflicts.append({
                    'room': room,
                    'day': day,
                    'conflict_ids': (prev[2], curr[2]),
                    'messages': "dipakai lebih dari 1 kelas"
                })
                penalty += 1
    
    return {
        'penalty': penalty,
        'violations': {
            'hard_teacher': len(teacher_conflicts),
            'hard_room': len(room_conflicts),
            'teacher_preference': len(teacher_preference_conflicts),
            'saturday': len(saturday_conflicts)
        },
        'teacher_conflicts': teacher_conflicts,
        'room_conflicts': room_conflicts,
        'teacher_preference_conflicts': teacher_preference_conflicts,
        'saturday_conflicts': saturday_conflicts
    }

population_size = 5
population = [create_random_schedule() for _ in range(population_size)]

for idx, schedule in enumerate(population):
    fitness = calculate_fitness(schedule)
    print(f"\nIndividu {idx+1}:")
    print(f"Total Penalty: {fitness['penalty']}")


Individu 1:
Total Penalty: 31.5

Individu 2:
Total Penalty: 37.0

Individu 3:
Total Penalty: 26.0

Individu 4:
Total Penalty: 42.5

Individu 5:
Total Penalty: 39.0


In [22]:
import random
import copy

# Fungsi bantu untuk memindahkan slot berdasarkan konflik
def move_slot(schedule, fitness_info):
    # Kumpulkan id_slot yang terlibat konflik (dosen dan ruangan)
    conflict_slots = []
    for conflict in fitness_info['teacher_conflicts']:
        conflict_slots.extend(conflict['conflict_ids'])
    for conflict in fitness_info['room_conflicts']:
        conflict_slots.extend(conflict['conflict_ids'])
    conflict_slots = list(set(conflict_slots))
    
    # Jika tidak ada konflik, tidak perlu melakukan pemindahan
    if not conflict_slots:
        return schedule

    # Pilih salah satu slot konflik secara acak
    slot_to_move_id = random.choice(conflict_slots)
    
    # Temukan slot yang akan dipindahkan
    conflict_slot = None
    for idx, slot in enumerate(schedule):
        if slot['id_slot'] == slot_to_move_id:
            conflict_slot = slot
            break
    if conflict_slot is None or conflict_slot['mata_kuliah'] is None:
        return schedule

    # Simpan data penugasan yang akan dipindahkan
    course_assignment = conflict_slot['mata_kuliah']
    teacher = conflict_slot['dosen']
    kelas = conflict_slot['kelas']
    sks = conflict_slot['sks']
    metode = conflict_slot['metode']
    day = conflict_slot['hari']

    # Temukan blok slot (berurutan pada hari yang sama) yang memuat penugasan tersebut
    indices = [i for i, s in enumerate(schedule) if s['mata_kuliah'] == course_assignment and s['hari'] == day]
    if not indices:
        return schedule
    indices.sort()
    
    # Hapus penugasan dari blok tersebut (set kembali ke kondisi kosong)
    for i in indices:
        schedule[i]['mata_kuliah'] = None
        schedule[i]['dosen'] = None
        schedule[i]['kelas'] = None
        schedule[i]['sks'] = None
        schedule[i]['metode'] = None
        # Jika offline, kita bisa reset ruangan ke nilai awal (diambil dari slot_generator)
        # Disini kita asumsikan slot_generator() menghasilkan slot dengan ruangan default.
        # Jika diperlukan, simpan informasi ruangan default secara terpisah.
    
    # Cari blok kandidat baru pada hari yang sama dengan jumlah slot = sks
    candidate_blocks = []
    for i in range(len(schedule) - sks + 1):
        block = schedule[i:i+sks]
        # Pastikan blok berada pada hari yang sama dan semua slot kosong
        if not all(s['mata_kuliah'] is None for s in block):
            continue
        if not all(s['hari'] == day for s in block):
            continue
        # Untuk offline, pastikan ruangan sama di seluruh blok
        if metode != 'Online' and not all(s['ruang'] == block[0]['ruang'] for s in block):
            continue
        candidate_blocks.append(block)
    
    # Jika ditemukan kandidat, pilih salah satu (misalnya kandidat pertama)
    if candidate_blocks:
        selected_block = candidate_blocks[0]
        for slot in selected_block:
            slot.update({
                "mata_kuliah": course_assignment,
                "dosen": teacher,
                "kelas": kelas,
                "sks": sks,
                "metode": metode,
                "ruang": "Online" if metode == 'Online' else selected_block[0]['ruang']
            })
    else:
        # Jika tidak ditemukan kandidat, kembalikan penugasan ke blok awal
        for i in indices:
            schedule[i].update({
                "mata_kuliah": course_assignment,
                "dosen": teacher,
                "kelas": kelas,
                "sks": sks,
                "metode": metode
            })
    return schedule

# Fungsi utama untuk proses update populasi dengan mekanisme GWO
def gwo_optimization(population, num_iterations=50):
    for iteration in range(num_iterations):
        # Evaluasi fitness untuk setiap individu (jadwal)
        fitnesses = [calculate_fitness(schedule)['penalty'] for schedule in population]
        best_index = fitnesses.index(min(fitnesses))
        best_schedule = population[best_index]
        
        print(f"Iterasi {iteration+1}, Best Penalty: {fitnesses[best_index]}")

        # Update setiap individu (kecuali solusi terbaik) dengan strategi pemindahan slot
        for idx, schedule in enumerate(population):
            if idx == best_index:
                continue  # Pertahankan solusi terbaik
            # Dapatkan informasi fitness dari individu saat ini
            fitness_info = calculate_fitness(schedule)
            # Lakukan pemindahan slot untuk mengurangi konflik
            new_schedule = move_slot(copy.deepcopy(schedule), fitness_info)
            # Jika solusi baru memiliki penalty yang lebih rendah, terima solusi baru
            if calculate_fitness(new_schedule)['penalty'] < fitness_info['penalty']:
                population[idx] = new_schedule
    return population

# Contoh inisialisasi populasi awal
population_size = 5
population = [create_random_schedule() for _ in range(population_size)]

# Tampilkan nilai fitness awal tiap individu
for idx, schedule in enumerate(population):
    fitness = calculate_fitness(schedule)
    print(f"\nIndividu {idx+1}:")
    print(f"Total Penalty: {fitness['penalty']}\n")

# Jalankan proses optimasi menggunakan GWO
population = gwo_optimization(population, num_iterations=50)

# Tampilkan nilai fitness akhir tiap individu
for idx, schedule in enumerate(population):
    fitness = calculate_fitness(schedule)
    print(f"\nIndividu {idx+1} setelah optimasi:")
    print(f"Total Penalty: {fitness['penalty']}")



Individu 1:
Total Penalty: 37.5


Individu 2:
Total Penalty: 35.0


Individu 3:
Total Penalty: 32.5


Individu 4:
Total Penalty: 43.0


Individu 5:
Total Penalty: 35.0

Iterasi 1, Best Penalty: 32.5
Iterasi 2, Best Penalty: 32.5
Iterasi 3, Best Penalty: 29.5
Iterasi 4, Best Penalty: 29.5
Iterasi 5, Best Penalty: 28.0
Iterasi 6, Best Penalty: 27.5
Iterasi 7, Best Penalty: 27.0
Iterasi 8, Best Penalty: 27.0
Iterasi 9, Best Penalty: 24.0
Iterasi 10, Best Penalty: 24.0
Iterasi 11, Best Penalty: 24.0
Iterasi 12, Best Penalty: 24.0
Iterasi 13, Best Penalty: 24.0
Iterasi 14, Best Penalty: 24.0
Iterasi 15, Best Penalty: 24.0
Iterasi 16, Best Penalty: 24.0
Iterasi 17, Best Penalty: 24.0
Iterasi 18, Best Penalty: 24.0
Iterasi 19, Best Penalty: 23.5
Iterasi 20, Best Penalty: 23.0
Iterasi 21, Best Penalty: 23.0
Iterasi 22, Best Penalty: 23.0
Iterasi 23, Best Penalty: 23.0
Iterasi 24, Best Penalty: 23.0
Iterasi 25, Best Penalty: 23.0
Iterasi 26, Best Penalty: 23.0
Iterasi 27, Best Penalty: 23.0
It

In [17]:
import random
import copy
import numpy as np

def teacher_conflict_in_candidate(new_schedule, candidate_start_idx, block_length, teacher, original_day):
    """
    Fungsi ini mengecek apakah candidate block (dari new_schedule[candidate_start_idx:candidate_start_idx+block_length])
    akan menyebabkan konflik bagi dosen tertentu pada hari original_day.
    Conflict terjadi jika interval waktu candidate block overlap dengan slot lain
    yang sudah terjadwal untuk dosen tersebut (selain candidate block yang kosong).
    Adjacent (tepat bersambung) dianggap tidak konflik.
    """
    candidate_block = new_schedule[candidate_start_idx:candidate_start_idx+block_length]
    candidate_start = time_to_minutes(candidate_block[0]['jam_mulai'])
    candidate_end = time_to_minutes(candidate_block[-1]['jam_selesai'])
    
    # Iterasi di seluruh slot pada hari yang sama
    for slot in new_schedule:
        if slot['hari'] != original_day:
            continue
        # Lewati slot kosong
        if slot['mata_kuliah'] is None:
            continue
        # Periksa hanya jadwal dosen yang sama
        if slot['dosen'] != teacher:
            continue
        # Dapatkan interval slot tersebut
        slot_start = time_to_minutes(slot['jam_mulai'])
        slot_end = time_to_minutes(slot['jam_selesai'])
        # Jika candidate block overlap dengan slot (tidak hanya bersambung)
        if candidate_start < slot_end and candidate_end > slot_start:
            return True
    return False

def perturb_schedule(schedule):
    """
    Fungsi ini melakukan modifikasi (perturbasi) pada schedule dengan cara memindahkan
    blok penjadwalan untuk satu kelas secara keseluruhan. Misalnya, jika kelas tersebut memiliki
    3 SKS (3 slot berurutan), maka seluruh 3 slot tersebut akan dipindahkan ke lokasi baru
    (dengan syarat slot kosong, berada pada hari yang sama, dan tidak menyebabkan konflik dosen).
    """
    new_schedule = copy.deepcopy(schedule)
    
    # Pilih secara acak salah satu slot yang sudah terisi (bukan kosong)
    assigned_indices = [i for i, slot in enumerate(new_schedule) if slot['mata_kuliah'] is not None]
    if not assigned_indices:
        return new_schedule
    idx = random.choice(assigned_indices)
    
    # Identifikasi blok (seluruh slot) yang memiliki informasi yang sama (misalnya, mata_kuliah, kelas, dosen)
    current_slot = new_schedule[idx]
    block_key = (current_slot['mata_kuliah'], current_slot['kelas'], current_slot['dosen'])
    
    # Cari batas awal dan akhir blok secara kontigu
    start_idx = idx
    end_idx = idx
    while start_idx > 0 and (new_schedule[start_idx-1]['mata_kuliah'],
                             new_schedule[start_idx-1]['kelas'],
                             new_schedule[start_idx-1]['dosen']) == block_key:
        start_idx -= 1
    while end_idx < len(new_schedule)-1 and (new_schedule[end_idx+1]['mata_kuliah'],
                                              new_schedule[end_idx+1]['kelas'],
                                              new_schedule[end_idx+1]['dosen']) == block_key:
        end_idx += 1
    block_length = end_idx - start_idx + 1

    # Cari candidate posisi baru (indeks awal) yang merupakan blok kosong dengan panjang yang sama
    # dan berada pada hari yang sama dengan blok aslinya.
    original_day = new_schedule[start_idx]['hari']
    candidate_positions = []
    for i in range(len(new_schedule) - block_length + 1):
        candidate_block = new_schedule[i:i+block_length]
        if all(slot['mata_kuliah'] is None for slot in candidate_block) and \
           all(slot['hari'] == original_day for slot in candidate_block):
            candidate_positions.append(i)
    
    if not candidate_positions:
        # Tidak ditemukan posisi yang valid untuk memindahkan blok
        return new_schedule

    # Filter candidate_positions berdasarkan pengecekan konflik dosen
    teacher = current_slot['dosen']
    filtered_candidates = []
    for pos in candidate_positions:
        if not teacher_conflict_in_candidate(new_schedule, pos, block_length, teacher, original_day):
            filtered_candidates.append(pos)
    
    if not filtered_candidates:
        # Jika tidak ada candidate yang menyebabkan konflik, kembalikan schedule tanpa perubahan.
        return new_schedule
    
    new_position = random.choice(filtered_candidates)
    
    # Simpan data blok yang akan dipindahkan
    block_data = [copy.deepcopy(new_schedule[j]) for j in range(start_idx, end_idx+1)]
    
    # Kosongkan blok asal (set nilai penjadwalan menjadi None)
    for j in range(start_idx, end_idx+1):
        new_schedule[j]['mata_kuliah'] = None
        new_schedule[j]['dosen'] = None
        new_schedule[j]['kelas'] = None
        new_schedule[j]['sks'] = None
        new_schedule[j]['metode'] = None
        # Biarkan atribut 'ruang', 'hari', 'jam_mulai', 'jam_selesai', dan 'id_slot' tetap
    
    # Tempel block_data ke posisi baru
    for k in range(block_length):
        new_schedule[new_position + k].update({
            "mata_kuliah": block_data[k]['mata_kuliah'],
            "dosen": block_data[k]['dosen'],
            "kelas": block_data[k]['kelas'],
            "sks": block_data[k]['sks'],
            "metode": block_data[k]['metode']
        })
        
    return new_schedule

# --- Implementasi Algoritma GWO untuk Penjadwalan ---

def gwo_optimization(num_wolves=10, max_iter=50):
    # Inisialisasi populasi (setiap wolf adalah schedule)
    population = [create_random_schedule() for _ in range(num_wolves)]
    fitnesses = [calculate_fitness(schedule)['penalty'] for schedule in population]
    
    # Identifikasi alpha (solusi terbaik), beta, dan delta
    sorted_indices = np.argsort(fitnesses)
    alpha = population[sorted_indices[0]]
    beta = population[sorted_indices[1]] if num_wolves > 1 else population[sorted_indices[0]]
    delta = population[sorted_indices[2]] if num_wolves > 2 else population[sorted_indices[0]]
    
    best_fitness = fitnesses[sorted_indices[0]]
    best_schedule = alpha
    
    for iteration in range(max_iter):
        for i in range(num_wolves):
            # Update solusi dengan operator perturbasi yang diadaptasi dari GWO
            candidate = perturb_schedule(population[i])
            candidate_fitness = calculate_fitness(candidate)['penalty']
            # Jika solusi candidate lebih baik, terima solusi tersebut
            if candidate_fitness < fitnesses[i]:
                population[i] = candidate
                fitnesses[i] = candidate_fitness
        
        # Update alpha, beta, delta
        sorted_indices = np.argsort(fitnesses)
        alpha = population[sorted_indices[0]]
        beta = population[sorted_indices[1]] if num_wolves > 1 else population[sorted_indices[0]]
        delta = population[sorted_indices[2]] if num_wolves > 2 else population[sorted_indices[0]]
        
        if fitnesses[sorted_indices[0]] < best_fitness:
            best_fitness = fitnesses[sorted_indices[0]]
            best_schedule = population[sorted_indices[0]]
            
        print(f"Iterasi {iteration+1}/{max_iter} - Best Fitness: {best_fitness}")
    
    return best_schedule, best_fitness

# --- Contoh Pemakaian ---
if __name__ == "__main__":
    best_schedule, best_fitness = gwo_optimization(num_wolves=10, max_iter=50)
    print("Solusi terbaik ditemukan dengan penalty:", best_fitness)
    
    fitness_result = calculate_fitness(best_schedule)
    
    print("\nDetail Konflik:")
    print("Konflik Dosen:", fitness_result['teacher_conflicts'])
    print("Konflik Ruangan:", fitness_result['room_conflicts'])
    
    for slot in best_schedule:
        if not slot['mata_kuliah']:
            continue
        print(f"Slot: {slot['id_slot']} - Hari: {slot['hari']} - Kelas: {slot['kelas']} - Dosen: {slot['dosen']} - Ruangan: {slot['ruang']} - Mata Kuliah: {slot['mata_kuliah']} - SKS: {slot['sks']} - Jam Mulai: {slot['jam_mulai']} - Jam Selesai: {slot['jam_selesai']}")


Iterasi 1/50 - Best Fitness: 22.0
Iterasi 2/50 - Best Fitness: 22.0
Iterasi 3/50 - Best Fitness: 22.0
Iterasi 4/50 - Best Fitness: 22.0
Iterasi 5/50 - Best Fitness: 22.0
Iterasi 6/50 - Best Fitness: 22.0
Iterasi 7/50 - Best Fitness: 22.0
Iterasi 8/50 - Best Fitness: 22.0
Iterasi 9/50 - Best Fitness: 22.0
Iterasi 10/50 - Best Fitness: 21.0
Iterasi 11/50 - Best Fitness: 21.0
Iterasi 12/50 - Best Fitness: 21.0
Iterasi 13/50 - Best Fitness: 21.0
Iterasi 14/50 - Best Fitness: 21.0
Iterasi 15/50 - Best Fitness: 21.0
Iterasi 16/50 - Best Fitness: 20.0
Iterasi 17/50 - Best Fitness: 20.0
Iterasi 18/50 - Best Fitness: 20.0
Iterasi 19/50 - Best Fitness: 20.0
Iterasi 20/50 - Best Fitness: 20.0
Iterasi 21/50 - Best Fitness: 19.0
Iterasi 22/50 - Best Fitness: 19.0
Iterasi 23/50 - Best Fitness: 19.0
Iterasi 24/50 - Best Fitness: 19.0
Iterasi 25/50 - Best Fitness: 19.0
Iterasi 26/50 - Best Fitness: 19.0
Iterasi 27/50 - Best Fitness: 19.0
Iterasi 28/50 - Best Fitness: 19.0
Iterasi 29/50 - Best Fitness: